<a href="https://colab.research.google.com/github/pulavartivinay/Face_Mask_Detection/blob/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
# start

In [121]:
# installs

In [122]:
# imports
import tensorflow as tf
from keras import backend as K
import cv2
from google.colab.patches import cv2_imshow
import glob
import itertools
import numpy as np
import random
import matplotlib.pyplot as plt
import pickle
import time
from PIL import Image

## Loading Dataset

In [123]:
# loading dataset
# https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset

!gdown --id 1NOzYPR3zAS8e2EFdRvjYpSBxg9RaDfxe #loading the dataset from google drive link to colab
!unzip /content/face-mask-12k.zip &> /dev/null #unzipping
!rm -rf /content/face-mask-12k.zip &> /dev/null #deleting the zip file
!rm -rf __MACOSX &> /dev/null

Downloading...
From: https://drive.google.com/uc?id=1NOzYPR3zAS8e2EFdRvjYpSBxg9RaDfxe
To: /content/face-mask-12k.zip
351MB [00:02, 170MB/s]


## Finding Correct Dimension for input

In [124]:
# finding the perfect crop size, for whole dataset
# https://stackoverflow.com/questions/19098104/python-opencv2-cv2-wrapper-to-get-image-size

def generate_image_paths(dir_path):
  files_png = glob.glob(dir_path + '/**/*.png', recursive=True)
  paths = list(files_png)
  return paths

train_dataset_path = "/content/face-mask-12k/Train"
val_dataset_path = "/content/face-mask-12k/Validation"

train_paths = generate_image_paths(train_dataset_path)
val_paths = generate_image_paths(val_dataset_path)

print("Images in Train Dataset ", len(train_paths)) #output: 10000
print("Images in Val Dataset", len(val_paths))   #output: 800

def find_avg_height_and_width(paths):
  sum_height = 0
  sum_width = 0
  for path in paths:
    image = cv2.imread(path)
    height, width, channels = image.shape
    sum_height += height
    sum_width += width
  avg_height = sum_height//len(paths)
  avg_width = sum_width//len(paths)
  return (avg_height, avg_width)

print("Train avg height and width", find_avg_height_and_width(train_paths)) #output: (153, 153)
print("Validation avg height and width", find_avg_height_and_width(val_paths)) #output: (152, 152)

Images in Train Dataset  10000
Images in Val Dataset 800
Train avg height and width (153, 153)
Validation avg height and width (152, 152)


## Hyperparamters

In [125]:
# hyper parameters
epochs = 10
batch_size = 16
img_dim = 224

## Preprocessing Data

In [126]:
# Preprocessing the data set into list of images
# https://keras.io/api/preprocessing/image/#image_dataset_from_directory-function

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/face-mask-12k/Train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_dim, img_dim),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="nearest",
    follow_links=False,
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/face-mask-12k/Validation",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size,
    image_size=(img_dim, img_dim),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="nearest",
    follow_links=False,
)

print("Train-dataset-length", len(train_dataset))
print("Validation-dataset-length", len(val_dataset))

Found 10000 files belonging to 2 classes.
Found 800 files belonging to 2 classes.
Train-dataset-length 625
Validation-dataset-length 50


In [127]:
# Model(Transfer Learning) (Later)

In [128]:
# Adding call Back Functions
# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Main Model Class with utility functions

In [164]:
# Main Model 

class MModel:
  def __init__(self, input_shape):
    self.input_shape = input_shape
    self.model = self.get_model()
    self.train_history = []
    self.val_history = []
    self.test_history = []

  def get_transfer_learning(self):
    mobilenetv2 = tf.keras.applications.MobileNetV2(input_shape=self.input_shape, include_top=False, weights="imagenet")
    mobilenetv2.trainable = False
    return mobilenetv2

  def get_model(self):
    mobilenetv2 = self.get_transfer_learning()

    inputs = tf.keras.Input(shape=self.input_shape)  #input layer
    rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255.)(inputs)
    base_model = mobilenetv2(rescale)
    conv1 = tf.keras.layers.Conv2D(filters=8, kernel_size=2, strides=(1, 1), padding="valid")(base_model)
    norm1 = tf.keras.layers.BatchNormalization()(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2), padding="valid")(norm1)
    drop1 = tf.keras.layers.Dropout(rate=0.1)(pool1)
    flat1 = tf.keras.layers.Flatten()(drop1)
    norm2 = tf.keras.layers.BatchNormalization()(flat1)
    dense1 = tf.keras.layers.Dense(32, activation='relu')(norm2)
    outputs = tf.keras.layers.Dense(2, activation='softmax')(dense1)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.summary()
    # plot_model(model, to_file='main_model_V040301.png') #convention for the model plot: main_model_VddMMXX where XX is the model number
    return model

  def compile(self, optimizer, loss, metrics):
    self.model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

  def train(self, train_dataset, train_batch_size, val_dataset, val_batch_size, epochs):
    self.train_history = self.model.fit(train_dataset, batch_size=train_batch_size, validation_data=val_dataset, validation_batch_size=val_batch_size, epochs=epochs, verbose=1)

  def evaluate(self, val_dataset, batch_size):
    self.val_history = self.model.evaluate(val_dataset, batch_size, verbose=1)

  def predict_single(self, image):
    """
      image dimensions should be (224, 224, 3)
    """
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    p = self.model.predict(input_arr, verbose = 0)
    return p

  def plot_loss_per_epoch(self):
    plt.plot(list(self.train_history.history.values())[0])
    plt.plot(list(self.train_history.history.values())[4])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'])
    plt.show()

  def plot_accuracy_per_epoch(self):
    plt.plot(list(self.train_history.history.values())[1])
    plt.plot(list(self.train_history.history.values())[5])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'])
    plt.show()

  def plot_f1_per_epoch(self):
    plt.plot(self.train_history.history['f1'])
    plt.title('model f1_score')
    plt.ylabel('f1')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'])
    plt.show()
  
  def plot_precision_per_epoch(self):
    plt.plot(list(self.train_history.history.values())[2])
    plt.plot(list(self.train_history.history.values())[6])
    plt.title('model precision_score')
    plt.ylabel('precision')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'])
    plt.show()

  def plot_recall_per_epoch(self):
    plt.plot(list(self.train_history.history.values())[3])
    plt.plot(list(self.train_history.history.values())[7])
    plt.title('model recall_score')
    plt.ylabel('recall')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'])
    plt.show()

  def write_model_with_weights_to_file(self, filepath):
    self.model.save(filepath)

In [165]:
# Creating instance of MModel Class

mmodel = MModel((224, 224, 3))

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling_17 (Rescaling)     (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 6, 6, 8)           40968     
_________________________________________________________________
batch_normalization_42 (Batc (None, 6, 6, 8)           32        
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 3, 3, 8)           0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 3, 3, 8)           0  

In [166]:
# Model Compilation
# https://keras.io/api/models/model_training_apis/

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
mmodel.compile(opt, "categorical_crossentropy", ['acc'])

## Training

In [167]:
# Training
# https://keras.io/api/models/model_training_apis/
epochs=100
mmodel.train(train_dataset, batch_size, val_dataset, -1, epochs)

Epoch 1/100
625/625 [==============================] - 47s 70ms/step - loss: 0.5158 - acc: 0.7497 - val_loss: 0.2381 - val_acc: 0.9125
Epoch 2/100
625/625 [==============================] - 43s 68ms/step - loss: 0.2173 - acc: 0.9179 - val_loss: 0.1464 - val_acc: 0.9500
Epoch 3/100
625/625 [==============================] - 43s 68ms/step - loss: 0.1584 - acc: 0.9451 - val_loss: 0.1048 - val_acc: 0.9625
Epoch 4/100
625/625 [==============================] - 43s 68ms/step - loss: 0.1196 - acc: 0.9611 - val_loss: 0.0812 - val_acc: 0.9737
Epoch 5/100
625/625 [==============================] - 43s 68ms/step - loss: 0.0934 - acc: 0.9724 - val_loss: 0.0653 - val_acc: 0.9800
Epoch 6/100
625/625 [==============================] - 43s 68ms/step - loss: 0.0818 - acc: 0.9748 - val_loss: 0.0541 - val_acc: 0.9837
Epoch 7/100
625/625 [==============================] - 43s 68ms/step - loss: 0.0701 - acc: 0.9787 - val_loss: 0.0456 - val_acc: 0.9850
Epoch 8/100
625/625 [==============================] - 

In [172]:
i=0
start = time.time()

'''measuring time'''
img = Image.open(val_paths[i])
img.resize((img_dim, img_dim), Image.NEAREST)
input_arr = tf.keras.preprocessing.image.img_to_array(img)
input_arr = np.array([input_arr])
p = mmodel.model.predict(input_arr, verbose = 1)

end = time.time()
print("time elapsed ", end - start)

1/1 [==============================] - 0s 19ms/step
time elapsed  0.05969738960266113


## Inferring Results

In [ ]:
# Predictions of Validation data set(Dev) - visualisation

i = 0
for batch in val_dataset:
  for i in range(batch_size):
    rgb_image = batch[0][i]
    label = batch[1][i].numpy()[0]

    bgr_image = cv2.cvtColor(rgb_image.numpy(), cv2.COLOR_RGB2BGR)
    cv2_imshow(bgr_image)

    print("Ground Truth", label)
    print("Prediction", mmodel.predict_single(rgb_image))
    i+=1
  if i>=10:
    break

## More inference

In [ ]:
# Drawing the train Loss, accuracy, f1, precision, recall of the model - visualising

mmodel.plot_loss_per_epoch()
mmodel.plot_accuracy_per_epoch()
# mmodel.plot_f1_per_epoch()
# mmodel.plot_precision_per_epoch()
# mmodel.plot_recall_per_epoch()

## Saving Model

In [173]:
# saving model to file

mmodel.write_model_with_weights_to_file('facemask12k_100epochs_mobilenetv2_224_rescaled_0.05s.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


## Testing Data